#  **Please Upvote if you find this notebook useful. This would be a great motivation for me**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import numpy
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Activation, BatchNormalization, MaxPooling2D, Dropout, Flatten, LeakyReLU
from keras.utils import np_utils
from keras import regularizers, models, layers
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_data = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test_data = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
print(train_data.shape)
print(test_data.shape)

In [ ]:
x_train = train_data.iloc[:,1:]
y_train = train_data.iloc[:,:1]

In [ ]:
x_train = x_train/255
x_train = x_train.to_numpy().reshape(-1,28,28,1)

In [ ]:
y_train = np_utils.to_categorical(y_train,10)  

In [ ]:
test_data = test_data/255
test_data = test_data.to_numpy().reshape(-1,28,28,1)

In [ ]:
print(x_train.shape)
print(y_train.shape)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train,y_train, test_size = 0.10, random_state=42 )

In [ ]:
print(x_train.shape) 
print(x_val.shape)
print(y_train.shape) 
print(y_val.shape)

In [ ]:
print(y_train)

In [ ]:
x_train[0:1].shape

In [ ]:
#a = np.array(train_data.iloc[:,0:1].values, dtype='float').reshape((28,28))  # or axis=1 
a = np.array(x_train[1900], dtype='float').reshape((28,28))
plt.imshow(a)
plt.show()
print(a.shape)
print(numpy.argmax(y_train[1900], axis=0))

In [ ]:
# model = Sequential()
# model.add(Dense(512, input_dim=784, activation='relu'))
# model.add(Dense(512, activation='relu'))
# model.add(Dense(10, activation='sigmoid'))

In [ ]:
# plt.imshow(x_reshape[0])
# print(x_reshape.shape)
# print(y_train.shape)

In [ ]:
# model=models.Sequential([
                     
#                      #cnn
#                     layers.Conv2D(30,(3,3),activation="relu",input_shape=(28,28,1)),
#                     layers.MaxPooling2D((2,2)),
     
#                     layers.Conv2D(64,(3,3),activation="relu"),
#                     layers.MaxPooling2D((2,2)),
#                      #dense_layer
#                      layers.Flatten(),
#                      layers.Dense(64,activation="relu"),
#                      layers.Dense(10,activation="softmax")

# ]
# )

In [ ]:
# x_train.shape[1:]

In [ ]:
# build the model

# number of hidden units variable 
# we are declaring this variable here and use it in our CONV layers to make it easier to update from one place
base_hidden_units = 32

# l2 regularization hyperparameter
weight_decay = 1e-4 

# instantiate an empty sequential model 
model = Sequential()

# CONV1
# notice that we defined the input_shape here because this is the first CONV layer. 
# we don’t need to do that for the remaining layers
model.add(Conv2D(base_hidden_units, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=(28,28,1)))
model.add(Activation('relu'))
model.add(BatchNormalization())

# CONV2
model.add(Conv2D(base_hidden_units, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# CONV3
model.add(Conv2D(2*base_hidden_units, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())

# CONV4
model.add(Conv2D(2*base_hidden_units, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

# CONV5
model.add(Conv2D(4*base_hidden_units, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())

# CONV6
model.add(Conv2D(4*base_hidden_units, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

# FC7
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

# print model summary
model.summary()

In [ ]:
model.compile(
            optimizer = "adam",
            loss ="categorical_crossentropy",
            metrics =["accuracy"]

)

In [ ]:
# print(x_reshape.shape)
# print(y_val_Cate.shape)

In [ ]:
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=100, batch_size=10)

In [ ]:
# model.fit(x_reshape, y_train.to_numpy(), validation_data=( x_val.to_numpy().reshape(-1,28,28,1), y_val.to_numpy()), epochs=10, batch_size=10)

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.9, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(x_val,  y_val, verbose=0)
print('Accuracy: %.4f' % (test_acc * 100.0))

In [ ]:
# test_reshape = test_data.to_numpy().reshape(-1,28,28,1)
# print(test_reshape.shape)
# test_reshape = test_reshape/255

In [ ]:
prediction = model.predict(test_data)
print(prediction)

In [ ]:
print(np.argmax(prediction[0]))
print(prediction[0])

In [ ]:
print(prediction[:1])
print(np.argmax(prediction, axis=1))

In [ ]:
print(len(prediction[0]))

In [ ]:
results = np.argmax(prediction, axis=1)
print(results)
print(len(results))

In [ ]:
actual_result = pd.DataFrame(results, columns= ['Label'])
print(actual_result)

In [ ]:
submission = pd.read_csv("../input/digit-recognizer/sample_submission.csv")
print(submission)

In [ ]:
submission['Label'] = actual_result['Label']
print(submission)

In [ ]:
submission['Label'].unique()

In [ ]:
submission.to_csv("submission.csv",index=False)

In [ ]:
!kaggle competitions submit -c digit-recognizer -f submission.csv -m "Message"